# Download and Import Dependacies

In [33]:
import pandas as pd

# Data Retrieval


In [34]:
from google.colab import drive
drive.mount('/drive')
%cd ..
%cd /drive/MyDrive/DIC_PROJECT
!ls

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).
/drive/My Drive
/drive/MyDrive/DIC_PROJECT
Olympic_Athlete_Biography.csv	   Olympic_Event_Results.csv  Olympic_Medal_Tally_History.csv
Olympic_Athlete_Event_Details.csv  Olympic_Games_Summary.csv  population_total_long.csv


In [35]:
Athlete_Events_Details = pd.read_csv('Olympic_Athlete_Event_Details.csv')
Event_Results = pd.read_csv('Olympic_Event_Results.csv')
Athlete_Biography = pd.read_csv('Olympic_Athlete_Biography.csv')
Medal_Tally = pd.read_csv('Olympic_Medal_Tally_History.csv')
Games_Summary = pd.read_csv('Olympic_Games_Summary.csv')
Population_Total = pd.read_csv('population_total_long.csv')

In [36]:
print(Athlete_Events_Details.shape)
print(Event_Results.shape)
print(Athlete_Biography.shape)
print(Medal_Tally.shape)
print(Games_Summary.shape)
print(Population_Total.shape)

(316834, 11)
(7394, 12)
(155861, 10)
(1807, 9)
(64, 11)
(12595, 3)


# Data Cleaning: Athlete_Events

In [37]:
Athlete_Events_Details.head(10)

,edition,edition_id,country_noc,sport,event,result_id,athlete,athlete_id,pos,medal,isTeamSport
0,1908 Summer Olympics,5,ANZ,Athletics,"100 metres, Men",56265,Ernest Hutcheon,64710,DNS,NaN,False
1,1908 Summer Olympics,5,ANZ,Athletics,"400 metres, Men",56313,Henry Murray,64756,DNS,NaN,False
2,1908 Summer Olympics,5,ANZ,Athletics,"800 metres, Men",56338,Harvey Sutton,64808,3 h8 r1/2,NaN,False
3,1908 Summer Olympics,5,ANZ,Athletics,"800 metres, Men",56338,Guy Haskins,922519,DNS,NaN,False
4,1908 Summer Olympics,5,ANZ,Athletics,"800 metres, Men",56338,Joseph Lynch,64735,DNS,NaN,False
5,1908 Summer Olympics,5,ANZ,Athletics,"800 metres, Men",56338,Henry Murray,64756,DNS,NaN,False
6,1908 Summer Olympics,5,ANZ,Athletics,"1,500 metres, Men",56349,Joseph Lynch,64735,5 h2 r1/2,NaN,False
7,1908 Summer Olympics,5,ANZ,Athletics,"1,500 metres, Men",56349,Charles Swain,79576,AC h3 r1/2,NaN,False
8,1908 Summer Olympics,5,ANZ,Athletics,"1,500 metres, Men",56349,Guy Haskins,922519,DNS,NaN,False
9,1908 Summer Olympics,5,ANZ,Athletics,"1,500 metres, Men",56349,George Blake,64619,DNS,NaN,False


In [38]:
print(Athlete_Events_Details.isnull().sum())

edition             0
edition_id          0
country_noc         0
sport               0
event               0
result_id           0
athlete             0
athlete_id          0
pos                 0
medal          272147
isTeamSport         0
dtype: int64


In [39]:
Athlete_Events_Details = Athlete_Events_Details.drop_duplicates()

In [40]:
Athlete_Events_Details = Athlete_Events_Details.drop(columns=['result_id', 'athlete', 'pos'])

In [41]:
Athlete_Events_Details['medal'].fillna('no medal', inplace=True)

<ipython-input-41-3bd0850ae94e>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  Athlete_Events_Details['medal'].fillna('no medal', inplace=True)


In [42]:
Athlete_Events_Details['country_noc'] = Athlete_Events_Details['country_noc'].str.strip().str.lower()
Athlete_Events_Details['sport'] = Athlete_Events_Details['sport'].str.strip().str.lower()
Athlete_Events_Details['event'] = Athlete_Events_Details['event'].str.strip().str.lower()
Athlete_Events_Details['edition'] = Athlete_Events_Details['edition'].str.strip().str.lower()

In [43]:
Athlete_Events_Details.head(10)

,edition,edition_id,country_noc,sport,event,athlete_id,medal,isTeamSport
0,1908 summer olympics,5,anz,athletics,"100 metres, men",64710,no medal,False
1,1908 summer olympics,5,anz,athletics,"400 metres, men",64756,no medal,False
2,1908 summer olympics,5,anz,athletics,"800 metres, men",64808,no medal,False
3,1908 summer olympics,5,anz,athletics,"800 metres, men",922519,no medal,False
4,1908 summer olympics,5,anz,athletics,"800 metres, men",64735,no medal,False
5,1908 summer olympics,5,anz,athletics,"800 metres, men",64756,no medal,False
6,1908 summer olympics,5,anz,athletics,"1,500 metres, men",64735,no medal,False
7,1908 summer olympics,5,anz,athletics,"1,500 metres, men",79576,no medal,False
8,1908 summer olympics,5,anz,athletics,"1,500 metres, men",922519,no medal,False
9,1908 summer olympics,5,anz,athletics,"1,500 metres, men",64619,no medal,False


In [44]:
Athlete_Events_Details['edition'] = Athlete_Events_Details['edition'].astype('category')
Athlete_Events_Details['isTeamSport'] = Athlete_Events_Details['isTeamSport'].astype(bool)

In [45]:
Athlete_Events_Details.head(10)

,edition,edition_id,country_noc,sport,event,athlete_id,medal,isTeamSport
0,1908 summer olympics,5,anz,athletics,"100 metres, men",64710,no medal,False
1,1908 summer olympics,5,anz,athletics,"400 metres, men",64756,no medal,False
2,1908 summer olympics,5,anz,athletics,"800 metres, men",64808,no medal,False
3,1908 summer olympics,5,anz,athletics,"800 metres, men",922519,no medal,False
4,1908 summer olympics,5,anz,athletics,"800 metres, men",64735,no medal,False
5,1908 summer olympics,5,anz,athletics,"800 metres, men",64756,no medal,False
6,1908 summer olympics,5,anz,athletics,"1,500 metres, men",64735,no medal,False
7,1908 summer olympics,5,anz,athletics,"1,500 metres, men",79576,no medal,False
8,1908 summer olympics,5,anz,athletics,"1,500 metres, men",922519,no medal,False
9,1908 summer olympics,5,anz,athletics,"1,500 metres, men",64619,no medal,False


In [46]:
Athlete_Events_Details[['year', 'olympic_type']] = Athlete_Events_Details['edition'].str.extract(r'(\d{4})\s+(summer|winter)\s+olympics')

print(Athlete_Events_Details['year'].head())
print(Athlete_Events_Details['olympic_type'].head())

0    1908
1    1908
2    1908
3    1908
4    1908
Name: year, dtype: object
0    summer
1    summer
2    summer
3    summer
4    summer
Name: olympic_type, dtype: object


In [47]:
Athlete_Events_Details['men'] = Athlete_Events_Details['event'].str.contains(r'\bmen\b', case=False).astype(int)
Athlete_Events_Details['women'] = Athlete_Events_Details['event'].str.contains('women', case=False).astype(int)

Athlete_Events_Details.head(10)

,edition,edition_id,country_noc,sport,event,athlete_id,medal,isTeamSport,year,olympic_type,men,women
0,1908 summer olympics,5,anz,athletics,"100 metres, men",64710,no medal,False,1908,summer,1,0
1,1908 summer olympics,5,anz,athletics,"400 metres, men",64756,no medal,False,1908,summer,1,0
2,1908 summer olympics,5,anz,athletics,"800 metres, men",64808,no medal,False,1908,summer,1,0
3,1908 summer olympics,5,anz,athletics,"800 metres, men",922519,no medal,False,1908,summer,1,0
4,1908 summer olympics,5,anz,athletics,"800 metres, men",64735,no medal,False,1908,summer,1,0
5,1908 summer olympics,5,anz,athletics,"800 metres, men",64756,no medal,False,1908,summer,1,0
6,1908 summer olympics,5,anz,athletics,"1,500 metres, men",64735,no medal,False,1908,summer,1,0
7,1908 summer olympics,5,anz,athletics,"1,500 metres, men",79576,no medal,False,1908,summer,1,0
8,1908 summer olympics,5,anz,athletics,"1,500 metres, men",922519,no medal,False,1908,summer,1,0
9,1908 summer olympics,5,anz,athletics,"1,500 metres, men",64619,no medal,False,1908,summer,1,0
